In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract
from glob import glob
import spacy
import re
import string

In [2]:
def cleanText(txt):
    whitespace = string.whitespace
    punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
model_ner = spacy.load('./output/model-best/')

In [127]:
image = cv2.imread('image/057.jpg')

tessData = pytesseract.image_to_data(image)

tessList = list(map(lambda x:x.split('\t'), tessData.split('\n')))
df = pd.DataFrame(tessList[1:],columns=tessList[0])
df.dropna(inplace=True) 
df['text'] = df['text'].apply(cleanText)


df_clean = df.query('text != "" ')
content = " ".join([w for w in df_clean['text']])
print(content)

doc = model_ner(content)

a hk 30342200 genericvics_haz, rev 2016-1212. please refer to www.vics.org for complete uldeline information. date 09/04/2019 bill of lading page j of 1 ship from bill of lading number 06026520001 780355 ame bar bakers clclerk@barbakers.com ityistate/zip los alamitos, ca 90720 sip 06026520001780355 ron y, kbco1719 ship to name fedex supply chain west . hocations appointment pt niner amar gh robinson railer number 335 ‘ aa rcs beat tubers oo 21 osha mba ar aches telto8 6 in veh ame kind snacks fedex supply chain pro number 303422960 ity/state/zip green bay, wi 54304 additional references freight charge terms freight charges are prepaid unless marked otherwise lpropaid jconect ¥jarapaty oh oo 8 ® 2 1 appt at 12 pm, please bring straps. calle lee location f a motor carrier, freight forwarder, broker or other transportation service provider accepts this shipment from an yone other than the shipper listed hereon, it agrees to seek payment of its harges exclusively from the entity from which

In [128]:
from spacy import displacy

In [129]:
displacy.serve(doc,style='ent')


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [130]:
displacy.render(doc,style='ent')

In [131]:
docjson = doc.to_json()
docjson.keys()

dict_keys(['text', 'ents', 'tokens'])

In [132]:
doc_text = docjson['text']

In [134]:
datafram_tokens = pd.DataFrame(docjson['tokens'])
datafram_tokens['token'] = datafram_tokens[['start','end']].apply(
    lambda x:doc_text[x[0]:x[1]] , axis = 1)
datafram_tokens.head(10)

,id,start,end,token
0,0,0,1,a
1,1,2,4,hk
2,2,5,13,30342200
3,3,14,29,genericvics_haz
4,4,29,30,","
5,5,31,34,rev
6,6,35,39,2016
7,7,39,40,-
8,8,40,44,1212
9,9,44,45,.


In [135]:
right_table = pd.DataFrame(docjson['ents'])[['start','label']]
datafram_tokens = pd.merge(datafram_tokens,right_table,how='left',on='start')

In [136]:
datafram_tokens.fillna('O',inplace=True)
datafram_tokens.head(10)

,id,start,end,token,label
0,0,0,1,a,O
1,1,2,4,hk,O
2,2,5,13,30342200,O
3,3,14,29,genericvics_haz,O
4,4,29,30,",",O
5,5,31,34,rev,O
6,6,35,39,2016,O
7,7,39,40,-,O
8,8,40,44,1212,O
9,9,44,45,.,O


In [137]:
df_clean['end'] = df_clean['text'].apply(lambda x: len(x)+1).cumsum() - 1 
df_clean['start'] = df_clean[['text','end']].apply(lambda x: x[1] - len(x[0]),axis=1)

In [138]:
dataframe_info = pd.merge(df_clean,datafram_tokens[['start','token','label']],how='inner',on='start')

In [139]:
dataframe_info.tail(10)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
371,5,1,46,2,2,2,134,3086,153,24,95,transportation,2311,2297,transportation,O
372,5,1,46,2,2,3,295,3085,109,23,96,according,2321,2312,according,O
373,5,1,46,2,2,4,411,3085,22,19,96,to,2324,2322,to,O
374,5,1,46,2,2,5,440,3083,35,20,96,the,2328,2325,the,O
375,5,1,46,2,2,6,483,3079,113,26,96,applicable,2339,2329,applicable,O
376,5,1,46,2,3,1,106,3117,114,23,84,egulations,2350,2340,egulations,O
377,5,1,46,2,3,2,228,3116,21,19,95,of,2353,2351,of,O
378,5,1,46,2,3,3,256,3116,34,20,95,the,2357,2354,the,O
379,5,1,46,2,3,4,300,3115,45,20,95,u.s.,2362,2358,u.s,O
380,5,1,46,2,3,5,355,3114,58,20,95,dot.,2367,2363,dot,O


In [140]:
bb_df = dataframe_info.query("label != 'O' ")
img = image.copy()

for x,y,w,h,label in bb_df[['left','top','width','height','label']].values:
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)
    
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.putText(img,str(label),(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)
    
    
cv2.imshow('Predictions',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [141]:
bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])
bb_df.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
9,5,1,2,1,1,7,684,312,149,24,88,www.vics.org,74,62,www.vics.org,DATE
14,5,1,3,1,1,1,65,353,72,27,96,date,114,110,date,DATE
15,5,1,3,1,1,2,157,352,167,32,70,09/04/2019,125,115,09/04/2019,DATE
25,5,1,3,1,2,3,1257,452,41,26,95,bill,167,163,bill,INVOICE_NO
26,5,1,3,1,2,4,1310,452,31,26,93,of,170,168,of,INVOICE_NO


In [142]:
class groupgen():
    def __init__(self):
        self.id = 0
        self.text = ''
        
    def getgroup(self,text):
        if self.text == text:
            return self.id
        else:
            self.id +=1
            self.text = text
            return self.id
        
grp_gen = groupgen()

In [143]:
bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)

In [144]:
bb_df[['left','top','width','height']] = bb_df[['left','top','width','height']].astype(int)
bb_df['right'] = bb_df['left'] + bb_df['width']
bb_df['bottom'] = bb_df['top'] + bb_df['height']

In [145]:
col_group = ['left','top','right','bottom','label','token','group']
group_tag_img = bb_df[col_group].groupby(by='group')

In [146]:
img_tagging = group_tag_img.agg({
    
    'left':min,
    'right':max,
    'top':min,
    'bottom':max,
    'label':np.unique,
    'token':lambda x: " ".join(x)
    
})

In [147]:
img_tagging

,left,right,top,bottom,label,token
group,,,,,,
1,65,833,312,384,[DATE],www.vics.org date 09/04/2019
2,1257,1874,428,481,[INVOICE_NO],bill of lading number 06026520001
3,111,119,2559,2580,[CUST_NAME],f


In [148]:
img_bb = image.copy()
for l,r,t,b,label,token in img_tagging.values:
    cv2.rectangle(img_bb,(l,t),(r,b),(0,255,0),2)
    cv2.putText(img_bb,str(label),(l,t),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)
cv2.imshow('',img_bb)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [149]:
def parser(text,label):
    if label=='CUST_NAME':
        text = text.lower()
        allow_special_char='.#_'
        text = re.sub(r'[^A-Za-z0-9{}]'.format(allow_special_char),'',text)
        
    elif label=='ADDRESS':
        text = text.lower()
        allow_special_char='@.,/\-_'
        text = re.sub(r'[^A-Za-z0-9{}]'.format(allow_special_char),'',text)
        
    elif label=='COMPANY_NAME':
        text = text.lower()
        allow_special_char='@.,/\-_'
        text = re.sub(r'[^A-Za-z0-9{}]'.format(allow_special_char),'',text)
        
    elif label=='INVOICE_NO':
        text = text.lower()
        text = re.sub(r'[\D]','',text)
        
    elif label=='PHONE':
        text = text.lower()
        allow_special_char = '-'
        text = re.sub(r'[^0-9{}]'.format(allow_special_char),'',text)
        
    elif label=='CUST_NO':
        text = text.lower()
        text = re.sub(r'[\D]','',text)
        
    elif label=='DATE':
        text = text.lower()
        allow_special_char = '/\-'
        text = re.sub(r'[^0-9{}]'.format(allow_special_char),'',text)
        
        
    return text
        

In [65]:
parser('date 12/27/2019','DATE')

'12/27/2019'

In [150]:
dataframe_info[['token','label']]

,token,label
0,a,O
1,hk,O
2,30342200,O
3,genericvics_haz,O
4,rev,O
...,...,...
376,egulations,O
377,of,O
378,the,O
379,u.s,O


In [151]:
info_array = dataframe_info[['token','label']].values
entities = dict(INVOICE_NO=[],CUST_NAME=[],ADDRESS=[],COMPANY_NAME=[],PHONE=[],CUST_NO=[],DATE=[])
previous = 'O'

for token,label in info_array:
    #print(token,label)
    bio_tag = label[0]
    label_tag = label[2:]
    text = parser(token,label_tag)
    if bio_tag in ('B','I'):
        if previous != label_tag:
            entities[label_tag].append(text)
        else:
            if bio_tag=="B":
                entities[label_tag].append(text)
            else:
                if label_tag in ('INVOICE_NO','CUST_NAME','ADDRESS','COMPANY_NAME','PHONE','CUST_NO','DATE'):
                    entities[label_tag][-1] = entities[label_tag][-1]+' '+text
                else:
                    entities[label_tag][-1] = entities[label_tag][-1]+text
                    
    previous = label_tag
    

In [152]:
print(entities)

{'INVOICE_NO': ['    06026520001'], 'CUST_NAME': ['f'], 'ADDRESS': [], 'COMPANY_NAME': [], 'PHONE': [], 'CUST_NO': [], 'DATE': ['', ' 09/04/2019']}
